In [1]:
import re
import pandas as pd
import xgboost as xgb
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
le = LabelEncoder()

/Users/MatthewBarnette/iron_yard_hw/titanic/.direnv/python-3.5.0/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [52]:
titanic_train = pd.read_csv("titanic/train.csv")
titanic_test = pd.read_csv("titanic/test.csv")
titanic_test_fit = pd.read_csv("titanic/test.csv")

In [53]:
titanic_train.Cabin.fillna('Steerage', inplace=True)
titanic_train = titanic_train.drop(['Name', 'Ticket'], axis=1)
titanic_train.Age = titanic_train.Age.fillna(int(titanic_train.Age.mean()))
titanic_train.Embarked = titanic_train.Embarked.fillna('U')


for row in range(len(titanic_train.Cabin)):
    if re.findall(r'^[A-Z, a-z]\d+', titanic_train.Cabin[row]):
        titanic_train.Cabin[row] = re.findall(r'^[A-Z, a-z]', titanic_train.Cabin[row])[0]


train, test = train_test_split(titanic_train, test_size = .4)

/Users/MatthewBarnette/iron_yard_hw/titanic/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [54]:
titanic_test.Cabin.fillna('Steerage', inplace=True)
titanic_test = titanic_test.drop(['Name', 'Ticket'], axis=1)
titanic_test.Age = titanic_test.Age.fillna(int(titanic_train.Age.mean()))
titanic_test.Embarked = titanic_test.Embarked.fillna('U')

for row in range(len(titanic_test.Cabin)):
    if re.findall(r'^[A-Z, a-z]\d+', titanic_test.Cabin[row]):
        titanic_test.Cabin[row] = re.findall(r'^[A-Z, a-z]', titanic_test.Cabin[row])[0]

/Users/MatthewBarnette/iron_yard_hw/titanic/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [55]:
titanic_test_fit.Cabin.fillna('Steerage', inplace=True)
titanic_test_fit = titanic_test_fit.drop(['Name', 'Ticket'], axis=1)
titanic_test_fit.Age = titanic_test_fit.Age.fillna(int(titanic_train.Age.mean()))
titanic_test_fit.Embarked = titanic_test_fit.Embarked.fillna('U')

for row in range(len(titanic_test_fit.Cabin)):
    if re.findall(r'^[A-Z, a-z]\d+', titanic_test_fit.Cabin[row]):
        titanic_test_fit.Cabin[row] = re.findall(r'^[A-Z, a-z]', titanic_test_fit.Cabin[row])[0]

/Users/MatthewBarnette/iron_yard_hw/titanic/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [56]:
for column in titanic_test.columns.values:
    if column == 'Sex' or column == 'Cabin' or column == 'Embarked':
        le.fit(titanic_test_fit[column])
        titanic_test[column] = le.transform(titanic_test[column])

for column in titanic_train.columns.values:
    if column == 'Sex' or column == 'Cabin' or column == 'Embarked':
        le.fit(titanic_train[column])
        train[column] = le.transform(train[column])

for column in titanic_train.columns.values:
    if column == 'Sex' or column == 'Cabin' or column == 'Embarked':
        le.fit(titanic_train[column])        
        test[column] = le.transform(test[column])

/Users/MatthewBarnette/iron_yard_hw/titanic/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MatthewBarnette/iron_yard_hw/titanic/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [57]:
survived_train = train['Survived']
train = train.drop(['Survived'], axis=1)

survived_test = test['Survived']
test = test.drop(['Survived'], axis=1)

In [58]:
dtrain = xgb.DMatrix(train.as_matrix(),
                     label=survived_train)
dtest = xgb.DMatrix(test.as_matrix(),
                    label=survived_test)

In [61]:
param = {'bst:max_depth':8, 'objective':'binary:logistic'}
param['nthread'] = 6
param['eval_metric'] = ['logloss']
evallist  = [(dtrain,'train'), (dtest,'eval')]
num_round = 500

In [62]:
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=3)

Will train until eval error hasn't decreased in 3 rounds.
[0]	train-logloss:0.531934	eval-logloss:0.581645
[1]	train-logloss:0.446473	eval-logloss:0.514687
[2]	train-logloss:0.382868	eval-logloss:0.478933
[3]	train-logloss:0.340647	eval-logloss:0.460820
[4]	train-logloss:0.309340	eval-logloss:0.449650
[5]	train-logloss:0.277643	eval-logloss:0.451634
[6]	train-logloss:0.257124	eval-logloss:0.448353
[7]	train-logloss:0.234978	eval-logloss:0.454577
[8]	train-logloss:0.223138	eval-logloss:0.457778
[9]	train-logloss:0.213090	eval-logloss:0.459709
Stopping. Best iteration:
[6]	train-logloss:0.257124	eval-logloss:0.448353



In [63]:
predictions = bst.predict(xgb.DMatrix(test.as_matrix()), output_margin=False)

In [66]:
pd.DataFrame(predictions)

,0
0,0.460327
1,0.120913
2,0.150319
3,0.222949
4,0.077094
5,0.217853
6,0.326861
7,0.106162
8,0.040231
9,0.664719
